## Depth First Search and Strongly Connected Components

### If you're using Datahub:
* Run the cell below **and restart the kernel if needed**

### If you're running locally:
You'll need to perform some extra setup.
#### First-time setup
* Install Anaconda following the instructions here: https://www.anaconda.com/products/distribution 
* Create a conda environment: `conda create -n cs170 python=3.10`
* Activate the environment: `conda activate cs170`
    * See for more details on creating conda environments https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html
* Install pip: `conda install pip`
* Install jupyter: `conda install jupyter`

#### Every time you want to work
* Make sure you've activated the conda environment: `conda activate cs170`
* Launch jupyter: `jupyter notebook` or `jupyter lab` 
* Run the cell below **and restart the kernel if needed**

In [46]:
# Install dependencies
!pip install -r requirements.txt --quiet

In [47]:
import otter
assert (
    otter.__version__ >= "5.5.0"
), "Please reinstall the requirements and restart your kernel."
import networkx as nx
import typing
import numpy as np
import tqdm
import pickle
grader = otter.Notebook("hw03.ipynb")

rng_seed = 0

In [48]:
# Load test cases
file_path = "generated_testcases.pkl"

# Load the variables from the pickle file
with open(file_path, "rb") as file:
    loaded_data = pickle.load(file)
file.close()
inputs, outputs = loaded_data

#### Representing graphs in code

There are multiple ways to represent graphs in code. In class we covered [adjacency matrices](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap3.pdf#page=2) and [adjacency lists](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap3.pdf#page=3). There is also the edge list representation, in which you store the edges in a single 1 dimensional list. In general for CS170 and in most cases, we choose to use the adjacency list representation since it allows us to efficiently search over a node's neighbors.

In many programming problems, verticies are typically labelled $0$ through $n-1$ for convenience (recall that arrays and lists in most languages begin at index 0). This allows us to represent an adjacency list using a list of lists that store ints. Given an edge list, the following code will create an adjacency list for an **unweighted directed graph**.

In [49]:
def generate_adj_list(n, edge_list):
    """
    args:
        n:int = number of nodes in the graph. The nodes are labelled with integers 0 through n-1
        edge_list:List[Tuple(int,int)] = edge list where each tuple (u,v) represents the directed 
            edge (u,v) in the graph
    return:
        A List[List[int]] representing the adjacency list 
    """
    adj_list = [[] for i in range(n)] 
    for u, v in edge_list:
        adj_list[u].append(v)
    for nodes in adj_list:
        nodes.sort()
    return adj_list

def draw_graph(adj_list):
    """Utility method for visualizing graphs

    args:
        adj_list (List[List[int]]): adjacency list of the graph given by generate_adj_list
    """
    G = nx.DiGraph()
    for u in range(len(adj_list)):
        for v in adj_list[u]:
            G.add_edge(u, v)
    nx.draw(G, with_labels=True)

## Q1.1) Reconstructing the DFS Path

In class we showed how to use DFS to check if there exists a path between two nodes, topologically sort nodes, and find SCCs. In those algorithms, pre and post numbers were used.

Here you'll implement a variation of DFS to print out the path between two nodes. In many problems, we want to be able to find the actual path between two nodes, not just determine if it exists. 

> **Task:** Compute a path from $s$ to $t$ using DFS and return the path as a list of nodes on that path. 

For example, the path ${s \to a \to b \to c \to t}$ corresponds to the list `[s, a, b, c, t]`. If no path exists, return the empty list `[]`.

You do not need to implement calculating pre and post numbers for this exercise.

*Hint:*
1. If you want to start with the recursive DFS implementation from DPV, you can use [mutable types or the `nonlocal` keyword](https://cs61a.org/study-guide/mutation/#local-state) to preserve state across recursive calls.
2. It may be helpful to maintain an extra data structure which tracks the previous node we visited each time we visit a new node. 

In [50]:
def dfs_path(adj_list, s, t):
    """
    args:
        adj_list:List[List] = an adjacency list 
        s:int = an int representing the starting node
        t:int = an int representing the destination node

    return: 
        a list of nodes starting with s and ending with t representing an s to t path if it exists. 
        Returns an empty list otherwise
    """

    vis = [False for _ in range(len(adj_list))]
    path = []

    def explore(adj_list, curr):
        """
        implements the explore subroutine from DPV, which is used in DFS. feel free to delete this 
        function and use an alternative implementation if you prefer.

        args:
            adj_list:List[List] = an adjacency list 
            curr:int = the node currently being traversed
        
        return:
            None
        """
        nonlocal vis
        path.append(curr)
        print("append:", curr)

        if curr == t:
            return True

        for nxt in adj_list[curr]:
            if not vis[nxt]:
                vis[nxt] = True
                if explore(adj_list, nxt):
                    return True
        
        print("pop:", path.pop())
        return False

    vis[s] = True
    if explore(adj_list, s):
        print("here")
        return path
    
    return []

    # # implement the dfs and path reconstruction here
    # ...

In [51]:
li = [[1],[0,2,10],[1,3,7],[2,4,8],[3,5,9],[4,6],[5],[2,8],[3,7],[4],[1]]
res = dfs_path(li, 10, 8)
print(res)

append: 10
append: 1
append: 0
pop: 0
append: 2
append: 3
append: 4
append: 5
append: 6
pop: 6
pop: 5
append: 9
pop: 9
pop: 4
append: 8
here
[10, 1, 2, 3, 8]


### Debugging

You can create sample tests in the following cells to help debug your solution. We provide a few small tests as an example, but they might not be comprehensive.

To add a new graph to the test, append a new edge list to `edge_lists` as shown in the next cell.  
__Remember that these edges are directed, so do not add both directions of an edge to the edge list.__

In [52]:
edge_lists = []
edge_lists.append([(0,1), (0,2), (1,2), (2,3), (3,4), (3,5), (4,5)])   # edge list of first graph
edge_lists.append([(0,1), (0,2), (1,2), (3,4), (3,5), (4,5)])          # edge list of second graph
# add any additional tests here

For each test case you also need to add a starting node $s$, a destination node $t$, and $n$ the number of nodes in the graph, add them to the following lists. 

In [53]:
s_list = []
s_list.append(0)  # s for first graph 
s_list.append(1)  # s for second graph 
# add any additional tests here

t_list = []
t_list.append(3)  # t for first graph
t_list.append(4)  # t for second graph
# add any additional tests here

n_list = []
n_list.append(6)  # n = 6 for first graph
n_list.append(6)  # n = 6 for second graph
# add any additional tests here

The following is a simplified version of the autograder, you may want to add more print statements or other debugging statements to check your function.

_Points:_ 2

In [54]:
import matplotlib.pyplot as plt
index = 1
for s, t, n, edge_list in zip(s_list, t_list, n_list, edge_lists):
    print("Testing graph:", index)
    index += 1
    
    adj_list_graph = generate_adj_list(n, edge_list) # function defined earlier
    
    path = dfs_path(adj_list_graph, s, t) 
    
    nx_graph = nx.DiGraph(edge_list)
    
    # uncomment the following to plot each graph
    '''
    nx.draw(nx_graph, with_labels=True)
    plt.title(f"Graph with {n} vertices and start node {s} and destination {t}")
    plt.show()
    '''
    
    if not nx.has_path(nx_graph,s,t):
        assert len(path) == 0, f"your dfs_path found an s-t path when there isn't one."
    else:
        # checks that the path returned is a real path in the graph and that it starts and ends 
        # at the right vertices
        assert nx.is_simple_path(nx_graph, path), f"your dfs_path did not return a valid simple path"
        assert path[0] == s, f"your dfs_path returned a valid simple path, but it does not start at node s"
        assert path[-1] == t, f"your dfs_path returned a valid simple path, but it does not end at node t"

print("Success")

Testing graph: 1
append: 0
append: 1
append: 2
append: 3
here
Testing graph: 2
append: 1
append: 2
pop: 2
pop: 1
Success


In [55]:
grader.check("q1.1")

 40%|████      | 20/50 [00:00<00:00, 195.42it/s]

append: 17
append: 4
append: 1
append: 6
append: 10
pop: 10
pop: 6
append: 8
append: 3
append: 0
append: 13
append: 7
append: 5
append: 11
append: 16
append: 19
append: 12
here
append: 20
append: 19
append: 7
append: 1
append: 2
append: 3
append: 4
append: 0
append: 11
append: 5
append: 8
append: 15
append: 10
here
append: 18
append: 14
append: 9
append: 13
append: 2
here
append: 6
append: 46
append: 37
append: 1
here
append: 17
append: 9
append: 11
append: 45
append: 12
append: 18
append: 2
append: 15
append: 21
append: 23
append: 31
append: 29
append: 40
append: 27
append: 32
append: 37
append: 28
append: 3
append: 39
append: 4
append: 14
append: 22
append: 51
pop: 51
append: 66
append: 44
append: 82
append: 7
pop: 7
append: 24
append: 68
append: 48
append: 5
append: 34
append: 38
append: 59
append: 33
append: 26
append: 13
append: 1
append: 52
append: 30
append: 65
append: 76
append: 0
append: 84
append: 16
append: 8
append: 49
append: 43
pop: 43
append: 86
append: 67
append: 79
app

 80%|████████  | 40/50 [00:00<00:00, 55.70it/s] 

append: 591
append: 27
append: 21
append: 275
append: 23
append: 12
append: 97
append: 89
append: 14
append: 62
append: 209
append: 171
append: 169
append: 47
append: 36
append: 28
append: 292
append: 42
append: 24
append: 26
append: 127
append: 107
append: 7
append: 55
append: 94
append: 158
append: 235
append: 65
append: 9
append: 50
append: 4
append: 111
append: 187
append: 136
append: 44
append: 17
append: 59
append: 2
append: 342
append: 18
append: 58
append: 305
append: 37
append: 273
append: 13
append: 31
append: 29
append: 143
append: 39
append: 219
append: 79
append: 119
append: 90
append: 101
append: 10
append: 129
append: 80
append: 3
append: 118
append: 51
append: 32
append: 274
append: 650
append: 53
append: 74
append: 237
append: 116
append: 25
append: 8
append: 120
append: 565
append: 63
append: 56
append: 11
append: 407
append: 434
append: 30
append: 253
append: 1
append: 57
append: 148
append: 488
append: 198
append: 281
append: 83
append: 300
append: 82
append: 285
ap

100%|██████████| 50/50 [00:01<00:00, 42.02it/s]

append: 248
append: 202
append: 63
append: 269
append: 215
append: 547
append: 32
append: 126
append: 8
append: 53
append: 596
append: 87
append: 23
append: 127
append: 148
append: 216
append: 250
append: 60
append: 411
append: 144
append: 51
append: 263
append: 34
append: 39
append: 131
append: 453
append: 128
append: 116
append: 108
append: 30
append: 393
append: 26
append: 97
append: 73
append: 20
append: 232
append: 178
append: 94
append: 236
append: 150
append: 66
append: 132
append: 106
append: 4
append: 162
append: 47
append: 49
append: 234
append: 45
append: 218
append: 35
append: 100
append: 201
append: 75
append: 64
append: 78
append: 67
append: 7
append: 84
append: 41
append: 245
append: 255
append: 68
append: 121
append: 57
append: 206
append: 122
append: 153
append: 155
append: 386
append: 10
append: 161
append: 253
append: 443
append: 311
append: 163
append: 101
append: 170
append: 117
append: 19
append: 72
append: 18
append: 61
append: 50
append: 44
append: 95
append: 85

q1.1 results: All test cases passed!

## 1.2) Pre and Post Numbers
In order to topologically sort or find strongly connected components, we need to be able to calculate pre and post numbers for each node. 

In this part, you will rework your implementation of DFS to allow it to generate pre and post order numbers for each node. It might be a good idea to copy/paste your solution from the previous part and modify it here. 

> **Task:** Implement a function that computes DFS pre and post numbers for each node in the graph.

To pass the autograder, your smallest preorder number should be 1. Your largest postorder number should be  $ 2 \times \text{(number of vertices)}$. Return two lists of tuples, a `pre` list should containing tuples `(node, pre-number)`, and a `post` list containing tuples `(node, post-number)`. 

Both lists should be ordered according to the pre/post number in the tuple. **You should not use any sorting functions to accomplish this!**

> **Reflect:** Why might returning pre/post numbers in this way be helpful for finding strongly connected components?

Feel free to delete the starter code and implement your own solution.

For this part, you can no longer assume that the entire graph is guaranteed to be reachable from some certain start node. How will this change your implementation?

Finally, break ties by choosing the node with the smallest number value. The autograder may fail implementations which are otherwise correct but break ties in a different way.

_Points:_ 2

In [78]:
def get_pre_post(adj_list):
    """
    args:
        adj_list:List[List[int]] = the adjacency list that represents our input graph
    return:
        List[Tuple(int, int)], List[Tuple(int, int)] representing the pre and post order values
            respectively. Each tuple should have a vertex as its first entry, and the pre/post order
            value as its second entry.
    """
    time = 1
    pre = []
    post = []

    # YOUR CODE HERE
    vis = [False for _ in range(len(adj_list))]

    def dfs(adj_list, now):
        nonlocal time
        
        pre.append((now, time))
        time += 1

        for nxt in adj_list[now]:
            if vis[nxt]:
                continue
            vis[nxt] = True
            dfs(adj_list, nxt)

        post.append((now, time))
        time += 1

    for node in range(len(adj_list)):
        if not vis[node]:
            vis[node] = True
            dfs(adj_list, node)        
    
    n = len(adj_list)
    # assert len(pre) == len(post) == n, f"{adj_list}\n{pre}"
    return pre, post

In [79]:
grader.check("q1.2")

100%|██████████| 110/110 [00:00<00:00, 2275.44it/s]


q1.2 results: All test cases passed!

## Q1.3 Identifying Tree, Forward, Back, Cross Edges

As we perform DFS traversals and create DFS trees and DFS forests within our graph, we would like to classify our edges according to how they appear in the resulting DFS forest. These classifications can provide us with insights about our graph. For example, the presence of a back edge $(u, v)$ tells us that we have a cycle within this graph that includes all the tree edges on the path from v to u and the back edge $(u, v)$.

>**Task 1.3**
>* Given the adjacency list of a graph, add each edge present in the edge set to the correct classification according the DFS traversal you implemented in part 1.
>* Don't modify the initialization of the edges_lookup dictionary.

_Points:_ 2

In [97]:
def categorize_edges(adj_list):
    """
    args:
        adj_list:List[List[int]] = the adjacency list that represents our input graph
    return:
        Dictionary({
            'tree': set(),
            'forward': set(),
            'cross': set(),
            'back': set()  
        }) where each set() contains the edges that belong to the corresponding edge type
    """
    edges_lookup = {
        'tree': set(),
        'forward': set(),
        'cross': set(),
        'back': set()
    }
    
    time = 1
    pre = []
    post = []

    nodes_info = {}

    for node in range(len(adj_list)):
        nodes_info[node] = {'pre': -1, 'post': -1, 'root': -1}

    def dfs(now):
        nonlocal time
        
        pre.append((now, time))
        nodes_info[now]['pre'] = time
        time += 1

        for nxt in adj_list[now]:
            if nodes_info[nxt]['pre'] == -1:  # Tree edge
                edges_lookup['tree'].add((now, nxt))
                nodes_info[nxt]['root'] = nodes_info[now]['root']  # 设置相同连通分量
                dfs(nxt)
            else:
                if nodes_info[nxt]['post'] == -1:  # Back edge
                    edges_lookup['back'].add((now, nxt))
                elif nodes_info[now]['pre'] < nodes_info[nxt]['pre']:  # Forward edge
                    edges_lookup['forward'].add((now, nxt))
                else:  # Cross edge
                    edges_lookup['cross'].add((now, nxt))

        post.append((now, time))
        nodes_info[now]['post'] = time
        time += 1


    for node in range(len(adj_list)):
        if nodes_info[node]['root'] == -1:
            nodes_info[node]['root'] = node
            dfs(node)

    return edges_lookup

In [98]:
li = [[1],[0,2,10],[1,3,7],[2,4,8],[3,5,9],[4,6],[5],[2,8],[3,7],[4],[1]]
print(categorize_edges(li))

{'tree': {(0, 1), (3, 8), (1, 2), (3, 4), (4, 9), (8, 7), (2, 3), (4, 5), (5, 6), (1, 10)}, 'forward': {(2, 7)}, 'cross': set(), 'back': {(2, 1), (6, 5), (4, 3), (5, 4), (10, 1), (8, 3), (7, 2), (1, 0), (3, 2), (7, 8), (9, 4)}}


In [99]:
grader.check("q1.3")

100%|██████████| 100/100 [00:00<00:00, 2279.98it/s]


q1.3 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [100]:
grader.export(pdf=False, force_save=True, run_tests=True)

<IPython.core.display.Javascript object>